In [176]:
from deck import *
from game import *
from player import *
from phases import Phases
import random
import torch


In [6]:
data = {
    "Silver": [],
    "Gold": []
}
# Tuple State, Legal_actions, action, reward

In [7]:

policies = [None,None]
wins=[0,0]
n_games = 1000


def get_move(index, state, move_indexes=None, move_strings=None):
    assert(move_indexes != None or move_strings != None), "at least one list must not be None {} {}".format(move_indexes,move_strings)

    if (move_indexes != None):
        length = len(move_indexes)
    else:
        length = len(move_strings)

    if policies[index] == None or length == 1:
        if move_indexes != None:
            move = random.sample(move_indexes,1)[0]
        else:
            move = random.sample(move_strings,1)[0]
    else:
        assert (False), "TODO"
        pass
    return move

In [38]:
# Simulate n_names times
for i in range(n_games):
    print("Starting game %d" % (i + 1))
    # Create game
    #game = Game([Player(get_8ed_core_gold_deck(), name="Gold"), Player(get_8ed_core_silver_deck(), name="Silver")])
    game = Game([Player(get_8ed_core_gold_deck(), name="Player1", deck_name="Gold"), Player(get_8ed_core_gold_deck(), name="Player2", deck_name="Gold")])
    game.start_game()

    game_data = {
        game.players[0].name: [],
        game.players[1].name: []
    }
    while not game.is_over():
        # Get player, valid moves
        player = game.player_with_priority
        move_indexes, move_strings = game.get_moves()
        state = game.get_board_string()

        # Skip COMBAT_DAMAGE_STEP_510_1c, no choices in damage assignment
        if game.current_phase_index == Phases.COMBAT_DAMAGE_STEP_510_1c:
            assert(len(move_indexes) == 1), "Should only be 1 value move"
            game.make_move(move_indexes[0],False)
            continue

        # Get move, may have to unroll action
        if isinstance(move_strings, ActionUnroller):
            unroller = move_strings
            while not unroller.is_done():
                moves = unroller.get_legal_moves()
                move_string = get_move(player.index,state, move_indexes=None, move_strings=moves)

                # Record Move
                if len(moves) > 1:
                    game_data[player.name].append([state, moves, move_string])

                #print(move_string)
                state = unroller.register_move(move_string)
            # Apply unrolled actions to game
            unroller.make_move()
        else:
            #print(state)
            move = get_move(player.index, state, move_indexes, move_strings)
            move_string = move_strings[move_indexes.index(move)]

            # Record Move
            if len(move_indexes) > 1:
                game_data[player.name].append([state, move_strings, move_string])

            #print(move_string)
            game.make_move(move, False)
        #move = mcts.uct(game, itermax=5)

    if game.players[1].has_lost:
        wins[0] += 1
        reward_0 = 1
        reward_1 = -1
        #print("State", state)
    elif game.players[0].has_lost:
        wins[1] += 1
        reward_0 = -1
        reward_1 = 1
        #print("State", state)
    # Add reward info
    name_0 = game.players[0].name
    name_1 = game.players[1].name
    len_0 = len(game_data[name_0])
    len_1 = len(game_data[name_1])
    for i in range(max(len_0, len_1)):
        if i <= len_0 - 1:
            game_data[name_0][i].append(reward_0)
        if i <= len_1 - 1:
            game_data[name_1][i].append(reward_1)
    # Add game_data to overall data
    for player in game.players:
        data[player.deck_name].append(game_data[player.name])
    
print("player 0 wins: %i, player 1 wins: %i" % (wins[0], wins[1]))


Starting game 1
Starting game 2
Starting game 3
Starting game 4
Starting game 5
Starting game 6
Starting game 7
Starting game 8
Starting game 9
Starting game 10
Starting game 11
Starting game 12
Starting game 13
Starting game 14
Starting game 15
Starting game 16
Starting game 17
Starting game 18
Starting game 19
Starting game 20
Starting game 21
Starting game 22
Starting game 23
Starting game 24
Starting game 25
Starting game 26
Starting game 27
Starting game 28
Starting game 29
Starting game 30
Starting game 31
Starting game 32
Starting game 33
Starting game 34
Starting game 35
Starting game 36
Starting game 37
Starting game 38
Starting game 39
Starting game 40
Starting game 41
Starting game 42
Starting game 43
Starting game 44
Starting game 45
Starting game 46
Starting game 47
Starting game 48
Starting game 49
Starting game 50
Starting game 51
Starting game 52
Starting game 53
Starting game 54
Starting game 55
Starting game 56
Starting game 57
Starting game 58
Starting game 59
Starti

In [9]:
np.array(game_data['Player1'])

array([['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Norwood Ranger$Mountain$Volcanic Hammer$Mountain$Forest$Hill Giant$Lava Axe$Forest$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Forest$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
        list(['Mountain', 'Mountain', 'Forest', 'Forest', 'Pass']),
        'Pass', 1],
       ['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Norwood Ranger$Mountain$Volcanic Hamme

In [39]:
len(data['Silver'])

0

In [40]:
data['Gold'][1][1]

['\n        player-color$ Silver$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Forest$Enormous Baloth$Stone Rain$Lava Axe$Mountain$Volcanic Hammer$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ Forest$\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['tapped_ability_0_Forest', 'Pass'],
 'tapped_ability_0_Forest',
 1]

In [19]:
data['Silver'][0][0]

['\n        player-color$ Silver$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Island$Island$Island$Giant Octopus$Vizzerdrix$Plains$Plains$Island$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Mountain$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Island', 'Island', 'Island', 'Plains', 'Plains', 'Island', 'Pass'],
 'Island',
 -1]

In [22]:
data['Gold'][0][0]

['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Ogre Taskmaster$Hill Giant$Volcanic Hammer$Norwood Ranger$Grizzly Bears$Mountain$Mountain$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Mountain', 'Mountain', 'Pass'],
 'Mountain',
 1]

In [51]:
wins

[411, 389]

In [60]:
len(data['Gold'][0])

76

In [63]:
len(data['Gold'][0][0][0])

692

In [65]:
max_per_game = -1
max_state_len = -1
# Count total amount of transitions
total_transitions = 0
for deck_name, deck_data in data.items():
    for game_data in deck_data:
        game_len = len(game_data)
        max_per_game = max(max_per_game, game_len)
        max_state_len = max(max_state_len, max([len(transition[0]) for transition in game_data]))
        total_transitions += game_len
print(total_transitions)
print(max_per_game)
print(max_state_len)

105274
148
1245


In [43]:
len(data['Gold'])

1600

In [106]:
flat_data = []
for deck_type, deck_data in data.items():
    for game in deck_data:
        flat_data.extend(game)

In [45]:
len(flat_data)

105274

In [35]:
gold_data = data['Gold']

In [107]:
flat_array = np.array(flat_data, dtype=object)

In [17]:
flat_array[0]

array(['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Mountain$Mountain$Goblin Raider$Mountain$Rampant Growth$Forest$Lava Axe$Forest$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Forest$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
       list(['Mountain', 'Mountain', 'Mountain', 'Forest', 'Forest', 'Pass']),
       'Forest', -1], dtype=object)

In [63]:
np.savez_compressed('./data/test_data', flat_array)

In [49]:
actions = set(flat_array[:, 2])

In [50]:
print(len(actions))

141


In [17]:
print(data['Gold'][0][0])

['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Mountain$Mountain$Forest$Goblin Raider$Mountain$Enormous Baloth$Goblin Raider$Enormous Baloth$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Mountain$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ', ['Mountain', 'Mountain', 'Forest', 'Mountain', 'Pass'], 'Mountain', 1]


In [66]:
import torchtext

In [19]:
def customer_tokenizer(text):
    return text.split("$")

In [20]:
def customer_tokenizer_whole(text):
    return [text]

In [68]:
import collections

In [71]:
counter_obj_actions = collections.Counter()

In [23]:
for action in actions:
    sub_actions = action.split("$")
    counter_obj_actions.update(sub_actions)

In [72]:
for action in actions:
    counter_obj_actions.update([action])

In [73]:
counter_obj_actions

Counter({'Hill Giant$1$block$Norwood Ranger$0': 1,
         'Norwood Ranger$0$block$Grizzly Bears$1': 1,
         'Norwood Ranger$0$block$Ogre Taskmaster$0': 1,
         'Norwood Ranger$0$block$Norwood Ranger$1': 1,
         'Spined Wurm$0$block$Grizzly Bears$1': 1,
         'Enormous Baloth$0$block$Enormous Baloth$1': 1,
         'Hill Giant$0$block$Enormous Baloth$0': 1,
         'Hill Giant$0': 1,
         'Hill Giant$1$block$Norwood Ranger$1': 1,
         'Hill Giant$0$block$Norwood Ranger$0': 1,
         'Grizzly Bears$0$block$Enormous Baloth$0': 1,
         'Refuse': 1,
         'Hill Giant$0$block$Goblin Raider$0': 1,
         'Hill Giant$1': 1,
         'Enormous Baloth$0$block$Enormous Baloth$0': 1,
         'Norwood Ranger$0': 1,
         'Enormous Baloth$1$block$Grizzly Bears$0': 1,
         'Spined Wurm$0$block$Hill Giant$1': 1,
         'Spined Wurm$0$block$Enormous Baloth$0': 1,
         'Enormous Baloth$0$block$Grizzly Bears$0': 1,
         'Hill Giant$1$block$Goblin Rai

In [77]:
vocab_actions = torchtext.vocab.Vocab(counter_obj_actions, min_freq=1, specials=["<unk>"])

In [78]:
for i in range(len(vocab_actions)):
    print(i, vocab_actions.itos[i])

0 <unk>
1 Enormous Baloth
2 Enormous Baloth$0
3 Enormous Baloth$0$block$Enormous Baloth$0
4 Enormous Baloth$0$block$Enormous Baloth$1
5 Enormous Baloth$0$block$Goblin Raider$0
6 Enormous Baloth$0$block$Goblin Raider$1
7 Enormous Baloth$0$block$Grizzly Bears$0
8 Enormous Baloth$0$block$Grizzly Bears$1
9 Enormous Baloth$0$block$Hill Giant$0
10 Enormous Baloth$0$block$Hill Giant$1
11 Enormous Baloth$0$block$Norwood Ranger$0
12 Enormous Baloth$0$block$Norwood Ranger$1
13 Enormous Baloth$0$block$Ogre Taskmaster$0
14 Enormous Baloth$0$block$Spined Wurm$0
15 Enormous Baloth$1
16 Enormous Baloth$1$block$Enormous Baloth$0
17 Enormous Baloth$1$block$Enormous Baloth$1
18 Enormous Baloth$1$block$Goblin Raider$0
19 Enormous Baloth$1$block$Goblin Raider$1
20 Enormous Baloth$1$block$Grizzly Bears$0
21 Enormous Baloth$1$block$Grizzly Bears$1
22 Enormous Baloth$1$block$Hill Giant$0
23 Enormous Baloth$1$block$Hill Giant$1
24 Enormous Baloth$1$block$Norwood Ranger$0
25 Enormous Baloth$1$block$Norwood Ran

In [85]:
counter_obj_states = collections.Counter()

In [96]:
def process_state(state):
    return state.replace(' ', '').replace('\n', '').split('$')

In [98]:
print(flat_data[0][0].replace(' ', '').replace('\n', '').split('$'))

['player-color', 'Gold', 'life', '20', 'opponent-life', '20', 'phase', 'Phases.MAIN_PHASE_PRE_COMBAT', 'hand', 'Mountain', 'Mountain', 'GoblinRaider', 'Mountain', 'RampantGrowth', 'Forest', 'LavaAxe', 'Forest', 'generic-debt', '0', 'manapool', 'White', '0', 'Blue', '0', 'Black', '0', 'Red', '0', 'Green', '0', 'Colorless', '0', 'opponent-cards', '6', 'self-lands-untapped', '', 'self-lands-tapped', '', 'opponent-lands-untapped', 'Forest', 'opponent-lands-tapped', '', 'self-creatures-untapped', '', 'self-creatures-tapped', '', 'opponent-creatures-untapped', '', 'opponent-creatures-tapped', '', 'attackers-blockers', 'None', 'damage-order', 'None', '']


In [218]:
max_state_token_len = -1
for transition in flat_data:
    state = process_state(transition[0])
    max_state_token_len = max(max_state_token_len, len(state))
    counter_obj_states.update(state)


In [129]:
vocab_states = torchtext.vocab.Vocab(counter_obj_states, min_freq=1, specials=["<unk>"])

In [130]:
type(vocab_states)

torchtext.vocab.Vocab

In [116]:
for i in range(len(vocab_states)):
    print(i, vocab_states.itos[i])

0 <unk>
1 Mountain
2 0
3 Forest
4 
5 None
6 GoblinRaider
7 Black
8 Blue
9 Colorless
10 Green
11 Red
12 White
13 attackers-blockers
14 damage-order
15 generic-debt
16 hand
17 life
18 manapool
19 opponent-cards
20 opponent-creatures-tapped
21 opponent-creatures-untapped
22 opponent-lands-tapped
23 opponent-lands-untapped
24 opponent-life
25 phase
26 player-color
27 self-creatures-tapped
28 self-creatures-untapped
29 self-lands-tapped
30 self-lands-untapped
31 HillGiant
32 EnormousBaloth
33 GrizzlyBears
34 NorwoodRanger
35 Phases.MAIN_PHASE_PRE_COMBAT
36 LavaAxe
37 20
38 OgreTaskmaster
39 SpinedWurm
40 1
41 Silver
42 Gold
43 2
44 3
45 VolcanicHammer
46 4
47 5
48 6
49 7
50 StoneRain
51 17
52 Phases.DECLARE_ATTACKERS_STEP
53 8
54 RampantGrowth
55 18
56 9
57 15
58 16
59 10
60 attacker:
61 blockedby:
62 14
63 12
64 13
65 11
66 Phases.DECLARE_BLOCKERS_STEP
67 19
68 -3
69 -4
70 Phases.DECLARE_BLOCKERS_STEP_509_2
71 -2
72 -1
73 -5
74 -7
75 -6
76 -8
77 -9
78 -11
79 0']
80 1']
81 ['GoblinRaider
82

In [221]:
trajectories = []

In [222]:
for game in data['Gold']:
    game_data = {'states': [], 'actions': [], 'rewards':[], 'timesteps':[], 'state_masks':[]}
    timestep = 0
    for trajectory in game:
        state = [vocab_states[word] for word in process_state(trajectory[0])]
        mask = [1] * len(state)
        padding_len = max_state_token_len - len(state)
        mask.extend([0] * padding_len)
        state.extend([0] * padding_len)

        game_data['state_masks'].append(mask)
        game_data['states'].append(state)
        game_data['actions'].append(vocab_actions[trajectory[2]])
        game_data['rewards'].append(trajectory[3])
        game_data['timesteps'].append(timestep)
        timestep +=1
    for key,value in game_data.items():
        if not isinstance(value, np.ndarray):
            game_data[key] = np.array(value)
    trajectories.append(game_data)

In [225]:
device = 'cuda'
num_trajectories = len(trajectories)
K=20
max_ep_len = max_per_game
act_dim = 1

In [226]:
max_state_token_len

107

In [235]:
def get_batch(batch_size=256, max_len=K):
        batch_inds = np.random.choice(
            np.arange(num_trajectories),
            size=batch_size,
            replace=True,
        )

        s, a, r, timesteps, mask, state_masks = [], [], [], [], [], []
        for i in range(batch_size):
            traj = trajectories[batch_inds[i]]
            si = random.randint(0, traj['rewards'].shape[0] - 1)

            # get sequences from dataset
            state_dim = max_state_token_len
            state_masks.append(traj['state_masks'][si:si + max_len].reshape(1, -1, state_dim))
            s.append(traj['states'][si:si + max_len].reshape(1, -1, max_state_token_len))
            a.append(traj['actions'][si:si + max_len].reshape(1, -1, act_dim))
            r.append(traj['rewards'][si:si + max_len].reshape(1, -1, 1))
            tlen = a[-1].shape[1]

            
            timesteps.append(np.arange(si, si + tlen).reshape(1, -1))
            timesteps[-1][timesteps[-1] >= max_ep_len] = max_ep_len-1  # padding cutoff
        
            # padding and state + reward normalization
            s[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), s[-1]], axis=1)
            state_masks[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), state_masks[-1]], axis=1)
            a[-1] = np.concatenate([np.ones((1, max_len - tlen, act_dim)) * -10., a[-1]], axis=1)
            r[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), r[-1]], axis=1)
            timesteps[-1] = np.concatenate([np.zeros((1, max_len - tlen)), timesteps[-1]], axis=1)
            mask.append(np.concatenate([np.zeros((1, max_len - tlen)), np.ones((1, tlen))], axis=1))

        s = torch.from_numpy(np.concatenate(s, axis=0)).to(dtype=torch.float32, device=device)
        state_masks = torch.from_numpy(np.concatenate(state_masks, axis=0)).to(dtype=torch.float32, device=device)
        a = torch.from_numpy(np.concatenate(a, axis=0)).to(dtype=torch.float32, device=device)
        r = torch.from_numpy(np.concatenate(r, axis=0)).to(dtype=torch.float32, device=device)
        timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).to(dtype=torch.long, device=device)
        mask = torch.from_numpy(np.concatenate(mask, axis=0)).to(device=device)

        return s, state_masks, a, r, timesteps, mask

In [236]:
batch = get_batch()

In [243]:
s, state_masks, a, r, timesteps, mask = batch